Load the dataset

In [5]:
import pandas as pd

df = pd.read_csv(r'C:\10 Kifia Tasks\Week-3\Insurance-Risk-Analytics\notebook\data\Data_insurance.txt', sep='|')


C:\Users\samuel\AppData\Local\Temp\ipykernel_19120\673940959.py:3: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\10 Kifia Tasks\Week-3\Insurance-Risk-Analytics\notebook\data\Data_insurance.txt', sep='|')


Add Risk Metrics

In [6]:
# Claim Frequency: 1 if claim occurred, 0 otherwise
df['HasClaim'] = df['TotalClaims'] > 0
df['ClaimFrequency'] = df['HasClaim'].astype(int)

# Claim Severity: average claim amount (only for policies with claims)
df['ClaimSeverity'] = df['TotalClaims'] / df['ClaimFrequency']
df['ClaimSeverity'] = df['ClaimSeverity'].replace([float('inf'), float('nan')], 0)

# Margin: profitability
df['Margin'] = df['TotalPremium'] - df['TotalClaims']


Save and Version the Updated Data

In [7]:
df.to_csv('data/updated_data.csv', index=False)


In [8]:
!dvc add data/updated_data.csv
!git add data/updated_data.csv.dvc
!git commit -m "feat: added claim frequency, severity, and margin metrics"



To track the changes with git, run:

	git add 'data\.gitignore' 'data\updated_data.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



[task-3 3eb0eff] feat: added claim frequency, severity, and margin metrics
 1 file changed, 5 insertions(+)
 create mode 100644 notebook/data/updated_data.csv.dvc


Data Segmentation for Hypothesis 1

Filter by Province

In [11]:
df_wc = df[df['Province'] == 'Western Cape']
df_gt = df[df['Province'] == 'Gauteng']


Stratified Sampling

In [12]:
# Define sample size per vehicle type (adjust as needed)
sample_size = df_wc['VehicleType'].value_counts().min()

# Sample from each vehicle type group
wc_sample = df_wc.groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)
gt_sample = df_gt.groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)


C:\Users\samuel\AppData\Local\Temp\ipykernel_19120\1497533518.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wc_sample = df_wc.groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)
C:\Users\samuel\AppData\Local\Temp\ipykernel_19120\1497533518.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gt_sample = df_gt.groupby('VehicleType').apply(lambda x: x.s

Combine and Save

In [13]:
balanced_df = pd.concat([wc_sample, gt_sample])
balanced_df.to_csv('data/balanced_province_sample.csv', index=False)


Track with DVC and Git

In [14]:
!dvc add data/balanced_province_sample.csv
!git add data/balanced_province_sample.csv.dvc
!git commit -m "feat: created stratified sample by VehicleType for province-based hypothesis testing"



To track the changes with git, run:

	git add 'data\balanced_province_sample.csv.dvc' 'data\.gitignore'


⠋ Checking graph




To enable auto staging, run:

	dvc config core.autostage true
[task-3 2575516] feat: created stratified sample by VehicleType for province-based hypothesis testing
 1 file changed, 5 insertions(+)
 create mode 100644 notebook/data/balanced_province_sample.csv.dvc


Verify Balance

In [15]:
pd.crosstab(balanced_df['Province'], balanced_df['VehicleType'])


VehicleType,Bus,Heavy Commercial,Light Commercial,Medium Commercial,Passenger Vehicle
Province,,,,,
Gauteng,51,51,51,51,51
Western Cape,51,51,51,51,51


Chi-Squared Test for Claim Frequency

In [16]:
from scipy.stats import chi2_contingency

contingency = pd.crosstab(balanced_df['Province'], balanced_df['HasClaim'])
chi2, p_freq, _, _ = chi2_contingency(contingency)

print(f"Chi-squared test for Claim Frequency: p-value = {p_freq:.4f}")


Chi-squared test for Claim Frequency: p-value = 1.0000


Interpretation

There is no statistically significant difference in claim frequency between Gauteng and Western Cape, based on the stratified sample. This suggests that province alone does not influence the likelihood of a claim.

T-Test for Claim Severity

In [21]:
from scipy.stats import ttest_ind

claims_wc = balanced_df[(balanced_df['Province'] == 'Western Cape') & (balanced_df['HasClaim'])]
claims_gt = balanced_df[(balanced_df['Province'] == 'Gauteng') & (balanced_df['HasClaim'])]

t_stat_sev, p_sev = ttest_ind(claims_wc['ClaimSeverity'], claims_gt['ClaimSeverity'], equal_var=False)

print(f"T-test for Claim Severity: p-value = {p_sev:.4f}")


T-test for Claim Severity: p-value = nan


T-Test for Margin


In [18]:
margin_wc = balanced_df[balanced_df['Province'] == 'Western Cape']['Margin']
margin_gt = balanced_df[balanced_df['Province'] == 'Gauteng']['Margin']

t_stat_margin, p_margin = ttest_ind(margin_wc, margin_gt, equal_var=False)

print(f"T-test for Margin: p-value = {p_margin:.4f}")


T-test for Margin: p-value = 0.7939


Interpretation

There is no statistically significant difference in profitability (Margin) between Gauteng and Western Cape. This suggests that regional segmentation based on margin alone is not justified for these two provinces.

Hypothesis 2: Risk Differences Between Zip Codes

Step 1: Select Two Zip Codes

In [22]:
df.groupby('PostalCode')['HasClaim'].sum().sort_values(ascending=False).head(10)


PostalCode
2000    486
122     210
299      67
8000     51
7784     50
470      44
1863     44
400      34
458      32
2196     32
Name: HasClaim, dtype: int64

Stratified Sampling by VehicleType

In [23]:
zip_a = df[df['PostalCode'] == 2000]
zip_b = df[df['PostalCode'] == 122]

sample_size = min(zip_a['VehicleType'].value_counts().min(), zip_b['VehicleType'].value_counts().min())

zip_a_sample = zip_a.groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)
zip_b_sample = zip_b.groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)

balanced_zip_df = pd.concat([zip_a_sample, zip_b_sample])


C:\Users\samuel\AppData\Local\Temp\ipykernel_19120\625838004.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_a_sample = zip_a.groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)
C:\Users\samuel\AppData\Local\Temp\ipykernel_19120\625838004.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_b_sample = zip_b.groupby('VehicleType').apply(lambda x:

Save and Track

In [24]:
balanced_zip_df.to_csv('data/balanced_zip_sample.csv', index=False)

# DVC and Git
!dvc add data/balanced_zip_sample.csv
!git add data/balanced_zip_sample.csv.dvc
!git commit -m "feat: stratified sample for zip code-based hypothesis testing"



To track the changes with git, run:

	git add 'data\balanced_zip_sample.csv.dvc' 'data\.gitignore'

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



[task-3 f16aabc] feat: stratified sample for zip code-based hypothesis testing
 1 file changed, 5 insertions(+)
 create mode 100644 notebook/data/balanced_zip_sample.csv.dvc


Chi-Squared Test for Claim Frequency

In [25]:
from scipy.stats import chi2_contingency

contingency = pd.crosstab(balanced_zip_df['PostalCode'], balanced_zip_df['HasClaim'])
chi2, p_freq_zip, _, _ = chi2_contingency(contingency)

print(f"Chi-squared test for Claim Frequency (Zip): p-value = {p_freq_zip:.4f}")


Chi-squared test for Claim Frequency (Zip): p-value = 0.4103


Use Mann-Whitney U Test for Claim Severity

In [31]:
from scipy.stats import mannwhitneyu

u_stat, p_mwu = mannwhitneyu(claims_zip_a_clean, claims_zip_b_clean, alternative='two-sided')
print(f"Mann-Whitney U test for Claim Severity (Zip): p-value = {p_mwu:.4f}")


Mann-Whitney U test for Claim Severity (Zip): p-value = 1.0000


Hypothesis 3: Margin Differences Between Zip Codes

T-Test for Margin

In [32]:
margin_zip_a = balanced_zip_df[balanced_zip_df['PostalCode'] == 2000]['Margin']
margin_zip_b = balanced_zip_df[balanced_zip_df['PostalCode'] == 122]['Margin']

t_stat_margin_zip, p_margin_zip = ttest_ind(margin_zip_a, margin_zip_b, equal_var=False)

print(f"T-test for Margin (Zip): p-value = {p_margin_zip:.4f}")


T-test for Margin (Zip): p-value = 0.2963


Hypothesis 4: Gender-Based Risk Differences

Segment the Data

In [33]:
group_male = df[df['Gender'] == 'Male']
group_female = df[df['Gender'] == 'Female']


In [55]:
# Determine the minimum sample size per VehicleType for both genders, capped at 200
sample_size = min(df[df['Gender'] == 'Male']['VehicleType'].value_counts().min(),
    df[df['Gender'] == 'Female']['VehicleType'].value_counts().min()
)

male_sample = df[df['Gender'] == 'Male'].groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)
female_sample = df[df['Gender'] == 'Female'].groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)

balanced_gender_df = pd.concat([male_sample, female_sample])


C:\Users\samuel\AppData\Local\Temp\ipykernel_19120\1532528992.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  male_sample = df[df['Gender'] == 'Male'].groupby('VehicleType').apply(lambda x: x.sample(n=sample_size, random_state=42)).reset_index(drop=True)
C:\Users\samuel\AppData\Local\Temp\ipykernel_19120\1532528992.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  female_sample = df[df['Gender'] == 'F

In [56]:
balanced_gender_df.to_csv('data/balanced_gender_sample.csv', index=False)

# DVC and Git
!dvc add data/balanced_gender_sample.csv
!git add data/balanced_gender_sample.csv.dvc
!git commit -m "feat: stratified sample by VehicleType for gender-based hypothesis testing"



To track the changes with git, run:

	git add 'data\balanced_gender_sample.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



On branch task-3
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   data/.gitignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../Data/
	Statistical_testing.ipynb
	dvc_local_storage/

no changes added to commit (use "git add" and/or "git commit -a")


Chi-Squared Test for Claim Frequency

In [57]:
from scipy.stats import chi2_contingency

contingency = pd.crosstab(balanced_gender_df['Gender'], balanced_gender_df['HasClaim'])
chi2, p_freq_gender, _, _ = chi2_contingency(contingency)
print(f"Claim Frequency p-value: {p_freq_gender:.4f}")


Claim Frequency p-value: 1.0000


T-Test for Margin

In [58]:
margin_male = balanced_gender_df[balanced_gender_df['Gender'] == 'Male']['Margin']
margin_female = balanced_gender_df[balanced_gender_df['Gender'] == 'Female']['Margin']

t_stat_margin_gender, p_margin_gender = ttest_ind(margin_male, margin_female, equal_var=False)
print(f"Margin p-value: {p_margin_gender:.4f}")


Margin p-value: 0.7981


Mann-Whitney U

In [59]:
from scipy.stats import mannwhitneyu

u_stat, p_mwu_gender = mannwhitneyu(claims_male, claims_female, alternative='two-sided')
print(f"Mann-Whitney U test for Claim Severity (Gender): p-value = {p_mwu_gender:.4f}")


Mann-Whitney U test for Claim Severity (Gender): p-value = 1.0000


# 📊 Hypothesis Testing Summary – Task 3

**Objective**: Evaluate risk and profitability differences across key customer segments to guide AlphaCare’s pricing and marketing strategy.

---

## ✅ Metrics Used
- **Claim Frequency**: % of policies with claims  
- **Claim Severity**: Average claim amount (if claimed)  
- **Margin**: TotalPremium − TotalClaims

---

## 🧪 Hypothesis Results

| Hypothesis | Segment | Metric | p-value | Decision | Insight |
|-----------|---------|--------|---------|----------|--------|
| H₀₁ | Province | Margin | 0.7939 | Fail to Reject | No profitability difference |
| H₀₂ | Zip Code | Claim Frequency | 0.4103 | Fail to Reject | No risk difference |
| H₀₂ | Zip Code | Claim Severity | 1.0000 | Fail to Reject | No cost difference |
| H₀₃ | Zip Code | Margin | 0.2963 | Fail to Reject | No margin difference |
| H₀₄ | Gender | Claim Severity | NaN | Invalid | Too few claims |

---

## 💼 Business Recommendations

- **No evidence** to support pricing changes based on province, zip code, or gender.
- **Claim Severity tests failed** due to low claim counts — future sampling should target higher-density segments.
- Focus future segmentation on **vehicle type, plan features, or behavioral data** for stronger predictive signals.
